### Chapter 1: System Modelling
---
# **Tutorial 1.a: Lagrange Mechanics of a Pendulum**
**Aim:** To derive the dynamic model of a double pendulum using the Lagrange method using Python's symbolic toolbox.

**Further watching:** <a href="https://www.youtube.com/watch?v=zhk9xLjrmi4"> MIT OCW lecture on Lagrange mechanics with examples. </a> {DEVNOTE: keep current attached resource, or is there an updated one?}

## **Contents**:
* [Chain Systems](#Chain-Systems)
* [Coordinate Frames](#Coordinate-Frames)
* [Lagrange Mechanics](#Lagrange-Mechanics)
* [Simulation](#Simulation)

# **Chain Systems**
We model robots as **chains** of rigid **links**. If the links follow one after the other, it's a serial chain (e.g. upper arm -> forearm). If there are branches where more than one *child* link follows from a single *parent* link, it's a parallel chain (e.g. torso -> left thigh OR right thigh).

Most of the systems we model will be open chains, i.e. they don't involve any loops. If two parallel branches connect together, they form a closed chain (e.g. a four bar linkage system, like Baleka's leg).

<img src = "chain2.png" width = "600">

### **Terminology:**
- **Chain** - a group of links that are attached to each other
- **Link** - a single rigid body within a chain system
- **Serial Chain** - a chain where each link attaches to the previous link
- **Parallel Chain** - a chain where multiple child links originate froma single parent link
- **Closed Chain** - parallel chains that join at both ends, forming a loop
- **Open Chain** - parallel chains that only join at the parent link

# **Coordinate Frames**
A coordinate *frame* consists of an origin and three perpendicular axes (\*if working in 3D). {DEVNOTE: explain coords in 2D as well}

In order to completely describe the location of an individual link in 3D space, you need six coordinates: the position $x,y,z$, and the orientation $\alpha,\beta,\gamma$ (the roll, pitch and yaw angles about these axes).

The **system space** is the set of all $6n$ coordinates you need to completely define the *pose* of an $n$-link system, when the position of all links is described *absolutely* (i.e. relative to the fixed world frame). This is a *maximal* coordinate system, since it needs the maximum number of coordinates.

An alternative approach - the one used with Lagrange mechanics - is to use a smaller system of *generalized coordinates*. For chain systems, these typically correspond to the **joint space** of the system: the positions of all the joints between links. These are relative coordinates: rather than describing the links in the world frame, the position of the next link is defined from the previous link.

<img src = "coordinate_systems.png" width = "400">

The minimum number of generalized coordinates you need to *completely and uniquely* describe the pose gives the number of *degrees of freedom (DOFs)* of the system.

## The Double Pendulum {DEVNOTE: single instead, or keep double??}
We'll be generating the dynamic model for the 2D double pendulum shown in the previous image.

It is a 2 DOF system, since it can be defined completely by the two joint angles, $\theta_1$ and $\theta_2$. (This is a *fixed base* system, since the first link is constrained such that it can only rotate. If it was a *floating base* system, where this link was free to move in all directions, we'd need an extra 2 DOFs - $x$ and $y$ - to completely describe the pose.)

Our generalized coordinate vector is therefore:
$$\mathbf{q} = \begin{pmatrix} \theta_1 \\ \theta_2 \end{pmatrix}$$

In [ ]:
# import libraries
import sympy as sym
import numpy as np

from IPython.display import display #for pretty printing
# display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>")) #ignore this, it'll be useful in tut 3

In [ ]:
# create symbolic variables

# system parameters
g = sym.symbols('g')
m1 ,  m2 = sym.symbols([ 'm1', 'm2']) # mass of links
l1 ,  l2 = sym.symbols([ 'l1', 'l2']) # length of links
In1, In2 = sym.symbols(['In1','In2']) # moment of intertia of links

# generalized coordinates
# {DEVNOTE: th1,dth1,ddth1 is better for expanding the system later, but th1,th2 .. dth1,dth2 etc is better for explaining pos,vel,acc}
X0, Y0 = sym.symbols(['X0','Y0']) # fixed position of first link
# #position, velocity, acceleration
# th1,dth1,ddth1 = sym.symbols(['\\theta_{1}','\dot{\\theta}_{1}','\ddot{\\theta}_{1}']) # link 1
# th2,dth2,ddth2 = sym.symbols(['\\theta_{2}','\dot{\\theta}_{2}','\ddot{\\theta}_{2}']) # link 2

th1  ,  th2 = sym.symbols([       '\\theta_{1}',       '\\theta_{2}']) #position
dth1 , dth2 = sym.symbols([ '\dot{\\theta}_{1}', '\dot{\\theta}_{2}']) #velocity
ddth1,ddth2 = sym.symbols(['\ddot{\\theta}_{1}','\ddot{\\theta}_{2}']) #acceleration

q   = sym.Matrix([  [th1],  [th2]]) #group into matrices
dq  = sym.Matrix([ [dth1], [dth2]])
ddq = sym.Matrix([[ddth1],[ddth2]])

display(ddq) #display prints it as cool latex stuff

**NOTE:** Above, it was stated that in 3D, you need 6 coordinates to completely describe the location of a link. In 2D, you need 3: the position $x,y$, and the orientation $\theta$. In other words, the 2D **system space** comprises $3n$ coordinates for an n-link system.

{DEVNOTE: rotations: cast to 2D and recast to 3D, or use my method?}
{DEVNOTE: functions, three options shown below}

In [ ]:
# STEP 1: system space coordinates written in terms of generalised coordinates
import sympy as sym
def Rotate(v, th):
    # the 2D system space coordinates are [x;y;th], so we need a rotation function that can work with this
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [          0,            0, 1]]) # rotation matrix, augmented because of the theta element of the vector
    S = sym.Matrix([[0],[0],[th]]) # angle of rotation
    return R*v + S # coordinates after rotation

def Offset(v1, v2):
    # this function is for applying a position-only offset (preserves the angle)
    vector_mask = sym.Matrix([[1],[1],[0]]) # gets rid of angle component of vector when multiplied elementwise
    return v1 + v2.multiply_elementwise(vector_mask)

# positions of each link in their own reference frames
r0 = sym.Matrix([[X0],[Y0],[0]] # position of the origin of the first link
r1_1 = sym.Matrix([[0],[-0.5*l1],[0]]) # read as: position r1, in frame 1
r2_2 = sym.Matrix([[0],[-0.5*l2],[0]]) # both point vertically down, with centres of mass halfway down the total length of the link

# positions of each link, moved into the inertial frame
r1_0 = Offset(Rotate(r1_1, th1), r0))

r2_1 = Offset(Rotate(r2_2, th2), r1_1)
r2_0 = Offset(Rotate(r2_1, th1), r0)

In [ ]:
# STEP 1: system space coordinates written in terms of generalised coordinates
import sympy as sym
def Rotate(v, th):
    # the 2D system space coordinates are [x;y;th], so we need a rotation function that can work with this
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [          0,            0, 1]]) # rotation matrix, augmented because of the theta element of the vector
    S = sym.Matrix([[0],[0],[th]]) # angle of rotation
    return R*v + S # coordinates after rotation

def get_XY(v):
    # this function is for applying a position-only offset (preserves the angle)
    vector_mask = sym.Matrix([[1],[1],[0]]) # gets rid of angle component of vector when multiplied elementwise
    return v.multiply_elementwise(vector_mask)

def frame_shift(v, offset_angle, offset_position):
    return Rotate(v, offset_angle) + get_XY(offset_position)

# positions of each link in their own reference frames
r0 = sym.Matrix([[X0],[Y0],[0]] # position of the origin of the first link
r1_1 = sym.Matrix([[0],[-0.5*l1],[0]]) # read as: position r1, in frame 1
r2_2 = sym.Matrix([[0],[-0.5*l2],[0]]) # both point vertically down, with centres of mass halfway down the total length of the link

# positions of each link, moved into the inertial frame
r1_0 = frame_shift(r1_1, th1, r0)

r2_1 = frame_shift(r2_2, th2, r1_1)
r2_0 = frame_shift(r2_1, th1, r0)

In [ ]:
# STEP 1: system space coordinates written in terms of generalised coordinates
import sympy as sym
def Rotate(v, th):
    # the 2D system space coordinates are [x;y;th], so we need a rotation function that can work with this
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [          0,            0, 1]]) # rotation matrix, augmented because of the theta element of the vector
    S = sym.Matrix([[0],[0],[th]]) # angle of rotation
    return R*v + S # coordinates after rotation

def getXY(v):
    # this function is for applying a position-only offset (preserves the angle)
    vector_mask = sym.Matrix([[1],[1],[0]]) # gets rid of angle component of vector when multiplied elementwise
    return v.multiply_elementwise(vector_mask)

# positions of each link in their own reference frames
r0 = sym.Matrix([[X0],[Y0],[0]] # position of the origin of the first link
r1_1 = sym.Matrix([[0],[-0.5*l1],[0]]) # read as: position r1, in frame 1
r2_2 = sym.Matrix([[0],[-0.5*l2],[0]]) # both point vertically down, with centres of mass halfway down the total length of the link

# positions of each link, moved into the inertial frame
r1_0 = Rotate(r1_1, th1) + getXY(r0)

r2_1 = Rotate(r2_2, th2) + getXY(r1_1)
r2_0 = Rotate(r2_1, th1) + getXY(r0)

# **Lagrange Mechanics**


# **Simulation**
